In [1]:
import numpy as np

In [2]:
from scipy.spatial.transform import Rotation

euler = np.array([0,  0, np.deg2rad(45)])
rot = Rotation.from_euler('XYZ', euler)
rot.as_matrix()

array([[ 0.70710678, -0.70710678,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [3]:
rot0 = Rotation.from_euler('XYZ', np.array([np.deg2rad(10), np.deg2rad(0), np.deg2rad(0)]))
rot1 = Rotation.from_euler('XYZ', np.array([np.deg2rad(0), np.deg2rad(20), np.deg2rad(0)]))
rot2 = Rotation.from_euler('XYZ', np.array([np.deg2rad(0), np.deg2rad(0), np.deg2rad(30)]))
print(rot0.as_quat())
print(rot1.as_quat())
print(rot2.as_quat())

[0.08715574 0.         0.         0.9961947 ]
[0.         0.17364818 0.         0.98480775]
[0.         0.         0.25881905 0.96592583]


In [14]:
m = np.array([1 ,2 ,3])
x, y, z = m

In [17]:
z

3

In [56]:
class Quaternion():
    def __init__(self, w, x, y, z):
        self.quat = np.array([w, x, y, z])

    @classmethod
    def from_array(cls, quat):
        return cls(quat[0], quat[1], quat[2], quat[3])

    @classmethod
    def from_rotation(cls, vec, theta):
        t = theta / 2
        s = np.sin(t)
        c = np.cos(t)
        return cls(c, vec[0]*s, vec[1]*s, vec[2]*s)    

    def __array__(self) -> np.ndarray:
        return self.quat
        
    def as_array(self):
        return self.quat

    def as_vector(self):
        return self.as_array().reshape(4, 1)

    def as_matrix(self):
        w, x, y, z = self.as_array()
        return np.array([
                    [w*w + x*x - y*y - z*z, 2*(x*y - w*z), 2*(w*y + x*z)], 
                    [2*(w*z + x*y), w*w - x*x + y*y - z*z, 2*(-w*x + y*z)], 
                    [2*(x*z - w*y), 2*(y*z + w*x), w*w - x*x - y*y + z*z], 
                ])

    def conjugate(self):
        w, x, y, z = self.as_array()
        return Quaternion(w, -x, -y, -z)

    def norm(self, ord=2):
        return np.linalg.norm(self.as_array(), ord=ord)

    def normalize(self):
        return self * (1.0 / self.norm())

    def inverse(self):
        return self.conjugate() * (1.0 / self.norm()**2)
    
    def __str__(self):
        return str(self.quat)

    def __repr__(self):
        return str(self.quat)

    def __pos__(self):
        return Quaternion.from_array(self.as_array())

    def __neg__(self):
        return Quaternion.from_array(-self.as_array())

    def __add__(self, other):
        if type(other) == Quaternion: 
            return Quaternion.from_array(self.as_array() + other.as_array())
        else:
            return Quaternion.from_array(self.as_array() + other)
    
    def __sub__(self, other):
        if type(other) == Quaternion: 
            return Quaternion.from_array(self.as_array() - other.as_array())
        else:
            return Quaternion.from_array(self.as_array() - other)
        
    def __mul__(self, other):
        if type(other) == Quaternion: 
            w0, x0, y0, z0 = self.as_array()
            w1, x1, y1, z1 = other.as_array()
            return Quaternion(
                    w0*w1 - x0*x1 - y0*y1 - z0*z1,
                    x0*w1 + w0*x1 - z0*y1 + y0*z1,
                    y0*w1 + z0*x1 + w0*y1 - x0*z1,
                    z0*w1 - y0*x1 + x0*y1 + w0*z1,
                )
        else:
            return Quaternion.from_array(self.as_array() * other)


    def get_matrix2(self):
        w, x, y, z = self.get_tuple()
        return np.array([
                    [1 - 2*y*y - 2*z*z, 2*x*y + 2*w*z, 2*x*z - 2*w*y], 
                    [2*x*y - 2*w*z, 1-2*x*x - 2*z*z, 2*y*z + 2*w*x], 
                    [2*x*z + 2*w*y, 2*y*z - 2*w*x, 1 - 2*x*x - 2*y*y], 
                ])

In [57]:
Quaternion(1, 2, 3, 4).conjugate().normalize().norm()

5.477225575051661

In [58]:
q1*q0

[ 0.03333333 -0.13333333 -0.3        -0.53333333]

In [40]:
str(q1.as_array())

'[ 0.03333333 -0.06666667 -0.1        -0.13333333]'

In [41]:
q0 = Quaternion.from_rotation([1, 0, 0], np.deg2rad(10))
q1 = Quaternion.from_rotation([0, 1, 0], np.deg2rad(20))
q2 = Quaternion.from_rotation([0, 0, 1], np.deg2rad(30))

In [6]:
print(q0.as_tuple())
print(q1.as_tuple())
print(q2.as_tuple())

(0.9961946980917455, 0.08715574274765817, 0.0, 0.0)
(0.984807753012208, 0.0, 0.17364817766693033, 0.0)
(0.9659258262890683, 0.0, 0.0, 0.25881904510252074)


In [7]:
print((q0*q1*q2).as_tuple())

(0.943714364147489, 0.12767944069578063, 0.14487812541736914, 0.2685358227515692)


In [8]:
rot = Rotation.from_euler('XYZ', np.array([np.deg2rad(10), np.deg2rad(20), np.deg2rad(30)]))
print(rot.as_quat())

[0.12767944 0.14487813 0.26853582 0.94371436]


In [9]:
print(rot.as_matrix())

[[ 0.81379768 -0.46984631  0.34202014]
 [ 0.54383814  0.82317294 -0.16317591]
 [-0.20487413  0.31879578  0.92541658]]


In [10]:
m = (q0*q1*q2).as_matrix()
print(m)

[[ 0.81379768 -0.46984631  0.34202014]
 [ 0.54383814  0.82317294 -0.16317591]
 [-0.20487413  0.31879578  0.92541658]]


In [11]:
v = np.array([[1], [0], [0]])

In [12]:
m @ v

array([[ 0.81379768],
       [ 0.54383814],
       [-0.20487413]])

In [13]:
rot.as_rotvec()

array([0.26026043, 0.29531805, 0.5473806 ])